In [1]:
# @title Libraries needed
# from google.colab import drive
import pandas as pd
# drive.mount('/content/drive')
modelfile1 = "./finetuned/simplet5-epoch-2-train-loss-1.1119-val-loss-1.3865"
modelfile2 = "./finetuned/simplet5-epoch-2-train-loss-0.8903-val-loss-1.1033"
csvfile1 = "./deploy1.csv"
#!pip install simplet5
from simplet5 import SimpleT5
model1 = SimpleT5()
model1.from_pretrained(model_type="t5", model_name="t5-base")
model2 = SimpleT5()
model2.from_pretrained(model_type="t5", model_name="t5-base")
# Load the fune tuned model
model1.load_model("t5",modelfile1 , use_gpu=False)
model2.load_model("t5",modelfile2 , use_gpu=False)

Global seed set to 42


In [2]:
# @title App
from ipywidgets import IntProgress,Textarea, Button, HTML, HBox, VBox, Dropdown, Output, Layout
from IPython.display import display
from datetime import datetime
pd.options.mode.chained_assignment = None
pd.options.display.max_colwidth = 300
btnLoad = Button(description='Summarise',disabled=False,button_style='success',icon='check')
ddmodel = Dropdown(options=['Model 1','Nodel 2'], description='Model',disabled=False,)
ddvenue = Dropdown(options=['OVO','Queen'], description='Venue',disabled=False,)
barload=IntProgress(value=0,min=0,max=10,description='Status',style={'bar_color': 'green'})
tarea1 = Textarea(placeholder='Data to summarise',description='',disabled=False,
                  layout=Layout(height="200px", width="100%"))
tarea2 = Textarea(placeholder='Result',description='',disabled=False,
                  layout=Layout(height="200px", width="100%"))
out01 = Output()
df = pd.read_csv(csvfile1)
def summ(b):
  global df,out01,barload,ddmodel,ddvenue,tarea1,tarea2,model1, model2
  barload.value=2
  if df is not None:
    out01.clear_output()
    with out01:
      display("Start: "+ str(datetime.now()))
      x = list(df.loc[df['VNAME'] == ddvenue.value, "etotal"])[0]
      if ddmodel.value == "Model 1":
        y = model1.predict("summarize: " + x)
      else:
        y = model2.predict("summarize: " + x)
      display("Selected venue: "+ ddvenue.value + " to summarise with the " + ddmodel.value)
      tarea1.value = x
      tarea2.value = y[0]
      display("End: "+ str(datetime.now()))
  barload.value=10
btnLoad.on_click(summ)
def instr():
  return HTML("""<h1>Welcome to this project</h1><h2>Text summarisation project</h2>""")
def app():
  ddvenue.options=list(df.VNAME.unique())
  display(VBox([instr(),HBox([ddvenue, ddmodel, btnLoad]), barload, out01,HBox([tarea1, tarea2])]))
app()